In [0]:
# TESTING........

from pyspark.sql.functions import col
#from pyspark.sql.types import DateType
from datetime import date, timedelta
from pyspark.sql.functions import when, col, udf
import reverse_geocoder as rg
from pyspark.sql.types import StringType


start_date = date.today() - timedelta(1)
file_path = "abfss://gold@abychen.dfs.core.windows.net/EarthquakeData"
file_path2 = "abfss://silver@abychen.dfs.core.windows.net/earthquake_events_silver"


df = spark.read.format("delta").load(file_path).filter(col("event_date")>"2025-04-14")
display(df)

In [0]:
df.write.format("delta").mode("append").save(file_path)

In [0]:
start_date = date.today() - timedelta(2)
print(start_date)

In [0]:
# TESTING........

from pyspark.sql.functions import col, current_timestamp, concat, lit, to_timestamp
#from pyspark.sql.types import DateType
from datetime import date, timedelta, datetime

start_date = datetime.now() - timedelta(days=1)
start_date2 = date.today()
file_path = "abfss://silver@abychen.dfs.core.windows.net/earthquake_events_silver"

'''
print(start_date)
df = spark.read.format("delta").load(file_path)
display(df)
'''
df = spark.read.format("delta").load(file_path)
df = df.withColumn("timestamp", to_timestamp(concat(col("event_date"), lit(" "), col("event_time")),"yyyy-MM-dd HH:mm:ss:SSS"))
#display(df.filter(df.timestamp >= start_date))
#display(df.filter(df.timestamp >= "2025-04-15"))

print(start_date2)

In [0]:
import requests, json
from pyspark.sql.functions import concat, lit, current_date, to_timestamp
from datetime import date, timedelta, datetime

start_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(1)
end_date = datetime.now().replace(hour=23, minute=59, second=59, microsecond=0) - timedelta(1)

print(start_date)
print(end_date)

url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    response = requests.get(url)
    # getting earthquake data for yesterday using API
    response.raise_for_status()
    # checking if response is successful

    data = response.json().get("features",[])
    # getting the features from the response in GeoJSON format

    if not data:
        print("No data received")
    else:
        #file_path = f"{bronze_adls}{start_date}_earthquake_data.json"
        json_data = json.dumps(data, indent=4)
        # converting geojson data to json
        print(json_data)
        #dbutils.fs.put(file_path, json_data, overwrite=True)
        #writing to adls
        #print(f"Data saved to {file_path}")

except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
